# Bert Partition

## Imports

In [1]:
!pip install transformers[torch] datasets evaluate wandb -q

DEPRECATION: nb-black 1.0.7 has a non-standard dependency specifier black>='19.3'; python_version >= "3.6". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nb-black or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import datasets
import pickle as pkl

from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BertForMaskedLM,
    get_scheduler,
    TrainingArguments,
    Trainer
)
import evaluate
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD

from tqdm.auto import trange, tqdm
import pandas as pd
from collections import defaultdict
import io

import wandb

2024-03-29 08:53:55.007516: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 08:53:55.834497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# minio handler to use remote data -- implements get and put methods with pickling option (view file)

from MinioHandler import MinioHandler

minio = MinioHandler()

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

### Parameters

In [3]:
wandb.login()

wandb: Currently logged in as: xenomirant. Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(
    project='grammar-bert-model1',
    entity='grammar-bert'
)

wandb: Currently logged in as: xenomirant (grammar-bert). Use `wandb login --relogin` to force relogin


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [6]:
SEQ_LEN = 64
BATCH_SIZE = 16
MLM_PROB = 0.15

#DATA_PATH = '/content/drive/MyDrive/nnlp/bert/biblioteka_prikluchenij_both_agr.csv'
DATA_PATH = "data/train_dataset.csv"
TEST_PATH = "data/tda_test.csv"
MODEL_NAME = 'DeepPavlov/rubert-base-cased'

### Data Preparation

In [8]:
#df = pd.read_csv(DATA_PATH, index_col = 0)
#df = df.drop(columns=['Unnamed: 0'])

In [9]:
# df

In [10]:
# tqdm.pandas()

# idx_init = df.initial.progress_apply(lambda x: x.replace(' ', ''))
# idx_pol = df.polypers.progress_apply(lambda x: x.replace(' ', ''))
# idx = -(idx_init == idx_pol)
# df['was_changed'] = idx

In [11]:
# df.to_csv(DATA_PATH, index=False)

### Train test splitting

In [12]:
# from sklearn.model_selection import train_test_split

# TEST_SIZE = 0.1

In [13]:
# df = pd.read_csv(DATA_PATH, index_col = 0)

In [14]:
# df

In [15]:
# train, test = train_test_split(df, test_size=TEST_SIZE, stratify = df["was_changed"])

In [16]:
# train.to_csv("data/train_bpa.csv")
# test.to_csv("data/test_bpa.csv")

### Pick items from test for TDA and homology computation

In [10]:
# df = pd.read_csv("data/test_bpa.csv", index_col = 0)

# df = df[df.was_changed]

In [15]:
# tda_data = df.sample(n = 250, random_state=42)

In [16]:
# tda_data.to_csv("tda_test.csv")

In [19]:
# minio.put_object(tda_data, save_name="data/tda_test.pkl", pickle=True)

ModularLM/data/tda_test.pkl: |####################| 0.08 MB/0.08 MB 100% [elapsed: 00:00 left: 00:00, 20.18 MB/sec]

In [24]:
# put everything to minio -- also possible to use default minio functions from Minio class

# minio.minio.fput_object(file_path="data/test_dataset.csv", bucket_name="public",
#                       object_name="ModularLM/data/test_dataset.csv")

## Dataset and collator

In [7]:
class PairsDataset(Dataset):
    def __init__(self,
                 tokenizer,
                 path=None,
                 data=None,
                 filter_same=True):
        if data is None:
            data = pd.read_csv(path, index_col = 0)
        self.dataset = data[(data.was_changed) | (not filter_same)].reset_index(drop=True)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text1 = self.tokenizer(self.dataset.loc[idx]['base'],
                     padding='max_length',
                     truncation=True,
                     max_length=SEQ_LEN)

        text2 = self.tokenizer(self.dataset.loc[idx]['polypers'],
                     padding='max_length',
                     truncation=True,
                     max_length=SEQ_LEN)

        return text1, text2

In [8]:
def collate_func(batch):
    batch = [data_collator.torch_call(item) for item in zip(*batch)]
    return batch

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer.pad_token = '[SEP]'
tokenizer.eos_token = '[SEP]'
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=MLM_PROB)

## MLM Train

In [20]:
# dt = datasets.Dataset.from_csv(DATA_PATH)
# dt = dt.remove_columns(['polypers', 'was_changed']).rename_column('initial', 'text')

In [21]:
# N_samples = 10**5

In [22]:
# def tokenize_function(example):
#     return tokenizer(example['text'], truncation=True)

# tok_dt = dt.select(range(N_samples)).map(tokenize_function, batched=True)
# tok_dt = tok_dt.train_test_split(test_size=100,
#                          shuffle=True,
#                          seed=42)

In [23]:
# training_args = TrainingArguments(
#     report_to = 'wandb',
#     output_dir='part1-model',
#     learning_rate=1e-3,
#     per_device_train_batch_size=16,
#     num_train_epochs=1,
#     # evaluation_strategy='steps',
#     # eval_steps=20,
#     logging_steps=20,
#     logging_first_step=True
# )

In [24]:
# model = BertForMaskedLM.from_pretrained(MODEL_NAME)
# model.to(device)
# pass

In [25]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tok_dt['train'],
#     # eval_dataset=tok_dt['test'],
#     tokenizer=tokenizer,
#     data_collator=data_collator
# )

In [26]:
# trainer.train()

## Model

In [10]:
def save_gradients(model, division_layer):
    layers = {}

    for name, param in model.named_parameters():
        if name.startswith(f'bert.encoder.layer.{division_layer}'):
            break
        if (param.requires_grad) and param.grad is not None:
            layers[name] = param.grad.detach()

    return layers

In [11]:
def change_gradients(model, layers, weight_mlm = 0.5, weight_cos = 1):
    for name, param in model.named_parameters():
        if name in layers:
            param.grad = weight_cos * param.grad + weight_mlm * layers[name]
        else:
            break

In [12]:
class CosWeight:
    def __init__(self, init_state=1, step=0.5):
        self.cur_state = init_state
        self.step = step

    def __mul__(self, other):
        res = self.cur_state * other
        self.cur_state = self.cur_state * self.step
        return res

    def __repr__(self):
        return str(self.cur_state)

In [13]:
class CosLoss:
    def __init__(self, vector=None, alpha=0):
        self.loss = nn.CosineEmbeddingLoss()
        self.target = torch.ones(BATCH_SIZE).to(model.device)
        self.alpha = alpha
        self.vector = vector

    def __call__(self, hid_ref, hid_cur, target):
        cos_loss = self.loss(hid_ref, hid_cur, target)
        if self.vector is not None:
            cos_loss += self.alpha * self.loss(self.vector, hid_ref - hid_cur, self.target)
        return cos_loss

In [14]:
def train(model, criteria, optimizer, lr_scheduler, data, hom_data, n_epochs=1,
          n_cosine=10, division_layer=4, weight_mlm=1,
          weight_cos=1, save_every_epoch=3, test_every=5000):

    # global mlm_losses, cosine_losses
    # change global loss tracking to local only -- for now it seems unnecessary
    global tda_save_dict

    tq_epoch = trange(n_epochs, desc='Epochs: ')
    tq_batch = tqdm(total=len(data))

    target = -torch.ones(BATCH_SIZE).to(model.device)
    grads = None
    
    # just initialization -- first few batches make no difference for tracking
    cos_loss = 0
    mlm_loss = 0
    hom_computed = 0
    ## TODO -- optimize for gradual saving of dict intead of accumulation


    # save necessary features to dict
    def save_tda_features(hom_data: torch.utils.data.DataLoader, save_dict: dict):

        nonlocal hom_computed
        hom_computed+=1
        base = []
        polypers = []
        
        with torch.no_grad():
            model.eval()
            for i, batch in tqdm(enumerate(hom_data)):
                # base embeddings after layer
                pred_base = model(**{k: v.to(model.device) for k, v in batch[0].items()},
                                  output_hidden_states=True)
                hid_ref = torch.mean(pred_base.hidden_states[division_layer], dim=1)
                base.extend(hid_ref.detach().cpu().numpy())

                # polypers embeddings after layer
                pred_new = model(**{k: v.to(model.device) for k, v in batch[1].items()},
                                  output_hidden_states=True)
                hid_cur = torch.mean(pred_new.hidden_states[division_layer], dim=1)
                polypers.extend(hid_cur.detach().cpu().numpy())
        
        save_dict[hom_computed]["base"] = base
        save_dict[hom_computed]["polypers"] = polypers
        return save_dict
        
    #########################################################
    # training loop 
    #########################################################
    for epoch in tq_epoch:
        tq_batch.reset()
        cosine_losses = [cos_loss]
        mlm_losses = [mlm_loss]

        for i, batch in enumerate(data):
            # save data for TDA
            if i % test_every == 0:
                tda_save_dict = save_tda_features(hom_data, tda_save_dict)
                # also a point of optimization
                minio.put_object(tda_save_dict, save_name="data/tda_save_dict.pkl", pickle=True)
                model.train()
                
            pred = model(**{k: v.to(model.device) for k, v in batch[0].items()},
                         output_hidden_states=True, )

            if i % n_cosine == 0:
                pred.loss.backward()
                grads = save_gradients(model, division_layer)
                optimizer.zero_grad()
                mlm_losses.append(pred.loss.detach().cpu())

            else:
                pred_new = model(**{k: v.to(model.device) for k, v in batch[1].items()},
                                 output_hidden_states=True)

                hid_ref = torch.mean(pred.hidden_states[division_layer], dim=1)
                hid_cur = torch.mean(pred_new.hidden_states[division_layer], dim=1)

                cos_loss = criteria(hid_ref, hid_cur, target)
                cos_loss.backward()

                change_gradients(model, grads, weight_mlm=weight_mlm,
                                 weight_cos=weight_cos)

                optimizer.step()
                optimizer.zero_grad()

                cosine_losses.append(cos_loss.detach().cpu())

                cos_loss = (sum(cosine_losses[-30:]) / len(cosine_losses[-30:])).item()
                mlm_loss = (sum(mlm_losses[-30:]) / len(mlm_losses[-30:])).item()

                wandb.log({"MLM loss": mlm_loss,
                           "Cosine loss": cos_loss})
                tq_batch.set_postfix({
                        'MLM loss': mlm_loss,
                        'Cosine loss': cos_loss
                    })

            tq_batch.update(1)
        if epoch+1 % save_every_epoch == 0:
            buffer = io.BytesIO()
            torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'mlm_loss': pred.loss,
                    'cos_loss': cos_loss
                            }, f=buffer)
                # TODO -- add custom hash to model instead of value
            minio.put_object(buffer.getvalue(), 
                             save_name=f"ckpt/model_save_epoch_{epoch}.pt")
    
    model.eval()

In [15]:
dt = PairsDataset(tokenizer, path=DATA_PATH)
dl = DataLoader(dt, batch_size=BATCH_SIZE, shuffle=True,
                collate_fn=collate_func, drop_last=True)

In [16]:
dt_tda = PairsDataset(tokenizer, path=TEST_PATH)
dl_tda = DataLoader(dt_tda, batch_size=BATCH_SIZE, shuffle=True,
                collate_fn=collate_func, drop_last=True)

In [17]:
model = BertForMaskedLM.from_pretrained(MODEL_NAME)
model.to(device)
pass

### Compute SVD on dataset

In [18]:
from SVDmatrix import GetSVD

get_svd = GetSVD(model=model, dataloader=dl, division_layer=3)

In [ ]:
get_svd.get_matrix()

In [59]:
len(get_svd.matrix)

538240

In [ ]:
get_svd.compute_svd()

In [45]:
minio.put_object(get_svd.matrix, save_name="3rd_layer_diff_matrix.pkl", pickle=True)

ModularLM/3rd_layer_diff_matrix.pkl: |####################| 1594.54 MB/1594.54 MB 100% [elapsed: 00:19 left: 00:00, 81.26 MB/sec]

In [56]:
matrix = minio.get_object("3rd_layer_diff_matrix.pkl", unpickle=True)

In [64]:
len(matrix)

538240

In [21]:
minio.put_object(get_svd.svd, save_name="3rd_layer_diff_svd.pkl", pickle=True)

ModularLM/3rd_layer_diff_svd.pkl: |####################| 1579.13 MB/1579.13 MB 100% [elapsed: 00:19 left: 00:00, 80.78 MB/sec]

In [32]:
# different solution (when library internal tobytes interface is implemented --
# this variant is prefered)

# minio.put_object(get_svd.svd[0].tobytes(), save_name="svd_test", pickle=False)

ModularLM/svd_test: |####################| 1576.88 MB/1576.88 MB 100% [elapsed: 00:19 left: 00:00, 81.94 MB/sec]

### Training

In [18]:
n_mlm = 2
n_cosine = 8
division_layer = 3
weight_mlm = 1
weight_cos = CosWeight(0.5)

In [19]:
for name, param in model.named_parameters():
    param.requires_grad = name.startswith(f"bert.encoder.layer.{division_layer}")

In [33]:
# vec = torch.normal(0.5,
#                    0.1,
#                    size=(768, ),
#                    requires_grad=False).repeat(BATCH_SIZE, 1)

In [20]:
optimizer = SGD(model.parameters(), lr=5e-4)
criterion = nn.CosineEmbeddingLoss()

num_epochs = 200
num_training_steps = num_epochs * len(dl)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [21]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [ ]:
tda_save_dict = defaultdict(dict)

train(model=model, criteria=criterion, optimizer=optimizer, 
      lr_scheduler=lr_scheduler, data=dl, hom_data=dl_tda,
      n_epochs=num_epochs, 
      n_cosine=n_cosine, division_layer=division_layer + 1,
      weight_mlm=weight_mlm, weight_cos=weight_cos, 
      save_every_epoch=5, test_every=5000)

Epochs:   0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/33640 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


0it [00:00, ?it/s]

ModularLM/data/tda_save_dict.pkl: |####################| 1.42 MB/1.42 MB 100% [elapsed: 00:00 left: 00:00, 1924.22 MB/sec]